# task 3

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *
import matplotlib as mpl 
mpl.rcParams['figure.dpi'] = 300    
from sklearn.metrics import make_scorer

In [20]:
#generic input training
train_descriptors = pd.read_csv("train_descriptors.csv")
train_mord3d = pd.read_csv("train_mord3d.csv")
train_morgan = pd.read_csv("train_morgan.csv")
train_rdk = pd.read_csv("train_rdk.csv")

#crystal structures
train_crystals = pd.read_csv("train_crystals.csv")
#atom distance data in crystal structures
train_distances = pd.read_csv("train_distances.csv")
#centroid distance data in crystal structures
train_centroid_distances = pd.read_csv("train_centroid_distances.csv")

test_descriptors = pd.read_csv("test_descriptors.csv")
test_mord3d = pd.read_csv("test_mord3d.csv")
test_morgan = pd.read_csv("test_morgan.csv")
test_rdk = pd.read_csv("test_rdk.csv")

#need to predict density of structure

#centrosymmetric, and density is here
train_crystals.keys()
train_centrosymmetric = train_crystals["is_centrosymmetric"]
train_density = train_crystals["calculated_density"]

#van der Waal contacts is in train distances

train_distances.keys()
train_VDW = train_distances["n_vdw_contacts"]

#centroid distances are here

train_centroid_distances.keys()

train_centroid = train_centroid_distances["mean"]

train_descriptors_full = train_descriptors.iloc[:, 3:-2].dropna(axis= 1, how="any")
test_descriptors_full = test_descriptors[train_descriptors_full.columns]

In [21]:
dropable_columns = []

In [22]:
#need to predict density of structure

#train_centrosymmetric
#train_density
#train_VDW 
#train_centroid 

In [23]:
train_descriptors_full = train_descriptors.iloc[:, 3:-2].dropna(axis= 1, how="any")
test_descriptors_full = test_descriptors[train_descriptors_full.columns]

for i in train_descriptors_full:
    N_uniques = train_descriptors_full[i].unique()
    if len(N_uniques) < 2:
        dropable_columns.append(i)

train_descriptors_full = train_descriptors_full.drop(columns=dropable_columns)
test_descriptors_full = test_descriptors_full.drop(columns=dropable_columns)

In [24]:
from sklearn.model_selection import train_test_split

X_train_final,X_test_final,y_train_final,y_test_final=  train_test_split(
            train_descriptors_full,train_centroid, test_size = 0.2,
            random_state= 42)

X_train,X_test,y_train,y_test=  train_test_split(
            X_train_final,y_train_final, test_size = 0.2,
            random_state= 42)

In [26]:
ss = preprocessing.StandardScaler()
ss.fit(train_descriptors_full)

X_train_ss = ss.transform(train_descriptors_full)
model_ridge_ss = linear_model.RidgeCV()
model_ridge_ss.fit(X_train_ss, train_centroid)
cv_scores = model_selection.cross_val_score(
    model_ridge_ss, 
    X_train_ss, 
    train_centroid, 
    scoring="neg_mean_absolute_error")

print("Average negative MAE:", round(np.mean(cv_scores), 3))
print("Standard deviation:", round(np.std(cv_scores), 3))

Average negative MAE: -0.271
Standard deviation: 0.005


In [29]:
print (-metrics.mean_absolute_error(train_centroid,model_ridge_ss.predict(X_train_ss)))

-0.2592770004716582


In [33]:
predictions = model_ridge_ss.predict(ss.transform(test_descriptors_full))

In [37]:
max(train_centroid)

12.321133088570932

In [38]:
min(train_centroid)

3.1282834054301007

In [39]:
max(predictions)

43.01394062979135

In [41]:
min(predictions)

3.8908674850217193

In [46]:
predictions[np.argwhere(predictions>16)[0]] = np.mean(train_centroid)

In [47]:
np.savetxt("task_3_predictions.csv",predictions)

### Data pre-processing

### Outputting predictions for automated checking 
Link: https://github.com/stevenkbennett/fons_datathon_testing

In [10]:
predictions = model.predict(test_descriptors_PCA)

In [11]:
np.savetxt("task_x_predictions.csv", predictions)
# Alternatively, if using pandas, you can use the following:
pd.DataFrame(predictions).to_csv("task_x_predictions.csv", header=False, index=False)